<a href="https://colab.research.google.com/github/JoaoNettoo/Dio-trilha-java-basico/blob/main/World_Crude_Oil_Reserves_Consumptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'oil-reserves-and-consumption-from-1995-to-2022:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5317141%2F8835796%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T212236Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D62045c6bbe477c2f674222ecc2098445ab3f1b5197ca9bad92de244ef63ff346cece300a59b58e351a6ef85088ad33ba688fdd4b993366bcb32f3e53adfdaa995bbaa950e9743c4d4b24684b61acb6c34725425ca6d64afbef28943adb0db8198d90bbbd8c36bfe8a3efedd0c21aad526e7be25eadb67e26fdb5e3bcc3ac952fc501b8d6095126eadf6d8f370fa58a78e33f3802a3ab68e44d8267a86b8473135fb12aba5537b305eb8c2c628b6c64a0c7a46c069720b91e99b26cb0a1fb698cfbc89eceeeef4e633a4990eb8fb4feeb093e91a341c4793d36621fc1beb76371f242dc92a383a33a355e97ee3c1122e0b8d794c92f93fb05c2a03046669275ca'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# World Crude Oil Reserves/Consumptions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## World Crude Oil Reserves by Country

In [ ]:
df0=pd.read_csv('/kaggle/input/oil-reserves-and-consumption-from-1995-to-2022/World Crude Oil Reserves from 1995 to 2021.csv')
display(df0[0:5])
df0.index=df0.iloc[:,0]

df=df0.T[1:]
display(df[0:5])

In [ ]:
drop_cols=[]
for j,col in enumerate(df.columns):
    if df.iloc[-1,j]==0:
        drop_cols+=[j]
#print(drop_cols)
df=df.drop(df.columns[drop_cols], axis=1)
display(df[0:5])

In [ ]:
df2=df.iloc[-1:,0:-1].T
df2=df2.sort_values('2021',ascending=False)
nations=df2.index.tolist()
print(nations)

In [ ]:
plt.figure(figsize=(12,6))
for column in nations[0:20]:
    plt.plot(df.index, df[column].astype(float), label=column)
plt.xlabel('Year')
plt.ylabel('TWh')
plt.title('Oil Reserves Top 20')
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.xticks(rotation='vertical')
plt.grid(True)
plt.show()

## Oil Consumption by Country

In [ ]:
df0=pd.read_csv('/kaggle/input/oil-reserves-and-consumption-from-1995-to-2022/Oil Consumption by Country 1965 to 2023.csv')
display(df0[0:5])
df0.index=df0.iloc[:,0]

df=df0.T[1:]
display(df[0:5])

In [ ]:
print(df0['Entity'].tolist())

In [ ]:
drop_cols=[]
for j,col in enumerate(df.columns):
    if df.iloc[-1,j]==0:
        drop_cols+=[j]
#print(drop_cols)
df=df.drop(df.columns[drop_cols], axis=1)
display(df[0:5])

In [ ]:
df2=df.iloc[-1:,0:-1].T
df2=df2.sort_values('2023',ascending=False)
nations=df2.index.tolist()

In [ ]:
#select 20 countries from 'nations'
nations2=[
 'United States',
 'China',
 'India',
 'Russia',
 'Japan',
 'South Korea',
 'Brazil',
 'Canada',
 'Germany',
 'Mexico',
 'Iran',
 'Indonesia',
 'Singapore',
 'France',
 'United Kingdom',
 'Spain',
 'Oceania',
 'Italy',
 'Thailand',
 'Turkey',
]

In [ ]:
plt.figure(figsize=(12,6))
for column in nations2:
    plt.plot(df.index, df[column].astype(float), label=column)
plt.xlabel('Year')
plt.ylabel('TWh')
plt.title('Oil Consumption Top 20')
plt.legend()
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.xticks(rotation='vertical')
plt.grid(True)
plt.show()